In [1]:
import torch

cuda_version_major = int(torch.version.cuda.split('.')[0])
cuda_version_major

11

In [2]:
# !wget https://raw.githubusercontent.com/airctic/icevision/master/install_colab.sh
# !bash install_colab.sh {cuda_version_major}

In [3]:
# # Restart kernel
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

In [4]:
# import fastai, icevision, torch
# print(fastai.__version__, icevision.__version__, torch.__version__)

In [5]:
import pandas as pd
import numpy as np
import timm
import torch
from torch import tensor
from torch import nn
from pathlib import Path
from collections import namedtuple 
from sklearn.model_selection import StratifiedKFold

In [6]:
from torch.cuda.amp import GradScaler,autocast
from torch.cuda.amp.grad_scaler import OptState

In [45]:
from icevision.core import ClassMap
from icevision.core.bbox import BBox
from icevision.core.tasks import Task
from icevision.models.ross.efficientdet.dataloaders import process_train_record, build_infer_batch
from icevision.models.ross.efficientdet import convert_raw_predictions
from icevision.core.record import BaseRecord
from icevision.core.record_components import (FilepathRecordComponent, InstancesLabelsRecordComponent, 
                                              BBoxesRecordComponent, ClassificationLabelsRecordComponent, 
                                              ScoresRecordComponent)
from icevision.parsers.parser import Parser
from icevision.data import FixedSplitter
from icevision.data.dataset import Dataset
from icevision import tfms
from icevision.models.utils import transform_dl
from icevision.metrics.coco_metric import COCOMetricType, COCOMetric
from icevision.models.utils import apply_batch_tfms, unload_records

In [8]:
from effdet import get_efficientdet_config, create_model_from_config, unwrap_bench, create_model, load_checkpoint
from effdet.bench import _post_process, _batch_detection
from effdet.config import set_config_readonly, set_config_writeable
from effdet.efficientdet import get_feature_info, BiFpn, BiFpnLayer, HeadNet, _init_weight, _init_weight_alt
from effdet.loss import DetectionLoss
from effdet.anchors import Anchors, AnchorLabeler

In [49]:
from fastai.tabular.core import TabularPandas, Categorify, FillMissing, Normalize
from fastai.tabular.model import get_emb_sz, TabularModel
from fastai.data.block import CategoryBlock
from fastai.vision.learner import create_head
from fastai.callback.hook import num_features_model
from fastai.callback.core import Callback
from fastai.callback.tracker import SaveModelCallback, CancelStepException
from fastai.optimizer import ranger
from fastai.losses import CrossEntropyLossFlat
from fastai.torch_core import params
from fastai.data.load import DataLoader
from fastai.learner import Learner
from fastai.torch_core import to_float,set_seed, find_bs
from fastai.metrics import flatten_check, Metric

from icevision.all import *

In [48]:
from fastcore.all import *

In [11]:
ImgSize = namedtuple("ImgSize", "width,height")

In [12]:
# from google.colab import drive
# drive.mount('/content/drive')

# Paths and preparing df

In [65]:
sz=512
bs=4
N_FOLDS = 2
set_seed(42)

In [66]:
# path = Path('/host_Data/Datasets/Kaggle-SIIM-Covid19/siim-covid_resize_1024px')
# path_img = path/'train'
path = Path('/host_Data/Datasets/Kaggle-SIIM-Covid19/siim-covid_resize_512px')
path_img = path / 'train'

In [67]:
# dcm_df = pd.read_csv(path/'COVID_dcm_metadata.csv')
# train_df = pd.read_csv(path/'COVID_train_df.csv')
# meta_df = pd.read_csv(path/'meta.csv')

dcm_df = pd.read_csv(path/'COVID_dcm_metadata.csv')
train_df = pd.read_csv(path/'COVID_train_df.csv')
meta_df = pd.read_csv(path/'meta.csv')

In [68]:
meta_df = meta_df[meta_df['split'] == 'train']

In [69]:
train_df['image_id'] = train_df['id'].apply(lambda x: f'{x.split("_image")[0]}')
train_df['classi_label'] = train_df[['Negative for Pneumonia', 'Typical Appearance','Indeterminate Appearance', 'Atypical Appearance']].idxmax(1)
train_df = train_df[['image_id', 'classi_label', 'label', 'StudyInstanceUID']]

In [70]:
dcm_df = dcm_df[['fname', 'Modality', 'PatientSex']]
dcm_df['image_id'] = dcm_df.fname.apply(lambda x: f'{x.split("/")[-1].split(".dcm")[0]}')
train_df = train_df.merge(dcm_df, on='image_id')

In [71]:
def get_bboxes(val):
    val = val.split(' ')
    if val[0] != 'none':
        bboxes = []
        for i, e in enumerate(val):
            if i % 6 == 2:
                bboxes.append([float(b) for b in val[i : i+4]])
        return bboxes
    else:
        return None

In [72]:
train_df['bboxes'] = train_df['label'].apply(lambda x: get_bboxes(x))
train_df = train_df.merge(meta_df, left_on='image_id', right_on='image_id')

In [73]:
train_df['Modality_n'] = train_df['Modality'].map({'DX':0, 'CR':1})
train_df['PatientSex_n'] = train_df['PatientSex'].map({'M':0, 'F':1})

In [74]:
def get_scaled_bbox(row, sz):
    x_scale = row.dim1/sz
    y_scale = row.dim0/sz
    
    rescaled_bboxes = []
    if row.bboxes != None:
        for bbox in row.bboxes:
            rescaled_bboxes.append( np.array(bbox) / np.array([x_scale, y_scale, x_scale, y_scale]))
            
        return rescaled_bboxes
    else:
        return None

In [75]:
train_df['rescaled_bboxes'] = train_df.apply(lambda row: get_scaled_bbox(row, 512), axis=1)
train_df['rescaled_bboxes'] = train_df['rescaled_bboxes'].apply(lambda x: [[0, 0,  1, 1]] if x == None else x)
train_df['box_label'] = train_df['label'].apply(lambda x: x.split(' ')[0])

In [76]:
train_df['fold'] = -1

strat_kfold = StratifiedKFold(n_splits=N_FOLDS, random_state=42, shuffle=True)
for i, (_, test_index) in enumerate(strat_kfold.split(train_df.image_id.values, train_df['classi_label'].values)):
    train_df.iloc[test_index, -1] = i
    
train_df['fold'] = train_df['fold'].astype('int')

# Parser and dataloader

In [77]:
# Here ObjectDetectionRecord from icevision and added ClassificationLabelsRecordComponent
def ObjectDetectionClassiRecord():
    return BaseRecord(
        (
            FilepathRecordComponent(),
            InstancesLabelsRecordComponent(),
            BBoxesRecordComponent(),
            ClassificationLabelsRecordComponent(),
            ScoresRecordComponent(task=Task('tabular')),
        )
    )
template_record = ObjectDetectionClassiRecord()

In [78]:
#custom parser from icevision - added classification class map and classification labels
class CustomParser(Parser):
    def __init__(self, template_record, img_dir, label_df):
        super().__init__(template_record=template_record)

        self.img_dir = img_dir
        self.df = label_df
        self.class_map = ClassMap(list(self.df['box_label'].unique()))
        self.classi_class_map = ClassMap(['Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance'], background=None)

    def __iter__(self):
        for o in self.df.itertuples():
            yield o

    def __len__(self) -> int:
        return len(self.df)

    def record_id(self, o):
        return o.image_id

    def parse_fields(self, o, record, is_new):
        if is_new:
            # record.set_filepath(f'{self.img_dir}/{o.image_id}.png')
            record.set_filepath(f'{self.img_dir}/{o.image_id}.jpg')
            record.set_img_size(ImgSize(width=sz, height=sz))
            record.detection.set_class_map(self.class_map)
            record.classification.set_class_map(self.classi_class_map)
            record.classification.set_labels([o.classi_label])
            record.tabular.set_scores([o.Modality_n, o.PatientSex_n])

        record.detection.add_bboxes([BBox.from_xyxy(float(bb[0]), float(bb[1]), float(bb[2]), float(bb[3])) for bb in o.rescaled_bboxes])
        record.detection.add_labels([o.box_label for _ in o.rescaled_bboxes])

In [79]:
# the fuction below is extension of build_train_batch. this is to add batch_classi targets["classi"] is for targets for classification
def build_train_batch_m(records):

    batch_images, batch_bboxes, batch_classes = zip(
        *(process_train_record(record) for record in records)
    )
    batch_classi = [record.classification.label_ids for record in records] #INSERTED CODE
    batch_tabular = [record.tabular.scores for record in records] #INSERTED CODE

    # convert to tensors
    batch_images = torch.stack(batch_images)
    batch_bboxes = [tensor(bboxes, dtype=torch.float32) for bboxes in batch_bboxes]
    batch_classes = [tensor(classes, dtype=torch.float32) for classes in batch_classes] 
    batch_classi = [tensor(classes, dtype=torch.float32) for classes in batch_classi] #INSERTED CODE
    batch_tabular = [tensor(scores, dtype=torch.int32) for scores in batch_tabular] #INSERTED CODE

    # convert to EffDet interface
    targets = dict(bbox=batch_bboxes, cls=batch_classes, tab=batch_tabular, classi=batch_classi)

    return (batch_images, targets), records

In [80]:
# the fuction below make use of build_train_batch_ to build valid_ds
def build_valid_batch_m(records):

    (batch_images, targets), records = build_train_batch_m(records)

    # convert to EffDet interface, when not training, dummy size and scale is required
    targets = dict(img_size=None, img_scale=None, **targets)

    return (batch_images, targets), records

In [81]:
def transform_dl_m(dataset, build_batch, batch_tfms=None, **dataloader_kwargs):
    """Creates collate_fn from build_batch by decorating it with apply_batch_tfms and unload_records"""
    collate_fn = apply_batch_tfms(build_batch, batch_tfms=batch_tfms)
    collate_fn = unload_records(collate_fn)
    return DataLoader(dataset=dataset, create_batch=collate_fn, **dataloader_kwargs)

In [82]:
def train_dl_m(dataset, batch_tfms=None, **dataloader_kwargs):
    return transform_dl_m(
        dataset=dataset,
        build_batch=build_train_batch_m,
        batch_tfms=batch_tfms,
        **dataloader_kwargs
    )

In [83]:
def valid_dl_m(dataset, batch_tfms=None, **dataloader_kwargs):
    return transform_dl_m(
                        dataset=dataset,
                        build_batch=build_valid_batch_m,
                        batch_tfms=batch_tfms,
                        **dataloader_kwargs
                        )

In [84]:
def get_dls(fold):
  
    train_idx = [f'{path_img}/{o}.jpg' for o in train_df[train_df["fold"] != fold].image_id]
    valid_idx = [f'{path_img}/{o}.jpg' for o in train_df[train_df["fold"] == fold].image_id]

    split = FixedSplitter([train_idx, valid_idx])
    parser = CustomParser(template_record, path_img, train_df)
    train_records, valid_records = parser.parse(autofix=True)

    train_tfms = tfms.A.Adapter([*tfms.A.aug_tfms(size=sz), tfms.A.Normalize()])
    valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(sz), tfms.A.Normalize()])

    
    train_ds = Dataset(train_records, train_tfms)
    valid_ds = Dataset(valid_records, valid_tfms)

    train_dl = train_dl_m(train_ds, batch_size=bs, num_workers=0, shuffle=True)
    valid_dl = valid_dl_m(valid_ds, batch_size=bs, num_workers=0, shuffle=False)

    return train_dl, valid_dl

In [85]:
#procs = [Categorify, FillMissing, Normalize]
#cat_names  = ['Modality', 'PatientSex']

#to = TabularPandas(train_df, procs, cat_names, cont_names=None, y_names='classi_label', y_block=CategoryBlock())
#emb_szs = get_emb_sz(to); emb_szs

# Preparing DetClassiBench + tabular data embeddings

In [86]:
#modeified effdet's EfficientDet to calculate classification output from backbone's output
class EfficientDetClassi(nn.Module):
    def __init__(self, config, pretrained_backbone=True, classi_class=None, alternate_init=False):
        super(EfficientDetClassi, self).__init__()
        self.config = config
        set_config_readonly(self.config)
        self.backbone = timm.create_model(
            config.backbone_name, features_only=True,
            out_indices=self.config.backbone_indices or (2, 3, 4),
            pretrained=pretrained_backbone, **config.backbone_args)
        feature_info = get_feature_info(self.backbone)
        self.fpn = BiFpn(self.config, feature_info)
        self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
        self.box_net = HeadNet(self.config, num_outputs=4)

        #INSERTED CODE STARTS
        backbone_features = num_features_model(nn.Sequential(*self.backbone.children()))
        self.vis_head = create_head(backbone_features, 100) #fastai's create_head
        self.tab = TabularModel([[3,3], [3,3]], n_cont=0, out_sz = 100, layers = [100, 250])
        self.final_head = nn.Sequential(
                                        nn.BatchNorm1d(200),
                                        nn.Dropout(0.25),
                                        nn.Linear(200, 100, bias=False),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(100),
                                        nn.Dropout(0.5),
                                        nn.Linear(100, 4)
                                            )  
        #INSERTED CODE ENDS
        
        '''
        self.classifier = nn.Sequential(
                                        nn.AdaptiveMaxPool2d(output_size=1),
                                        nn.Flatten(),
                                        nn.BatchNorm1d(backbone_features),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(backbone_features, 512),
                                        nn.ReLU(inplace=True),
                                        nn.BatchNorm1d(512),
                                        nn.Dropout(p=0.25, inplace=False),
                                        nn.Linear(512, classi_class), 
        )
  
        '''
        for n, m in self.named_modules():
            if 'backbone' not in n:
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def reset_head(self, num_classes=None, aspect_ratios=None, num_scales=None, alternate_init=False):
        reset_class_head = False
        reset_box_head = False
        set_config_writeable(self.config)
        if num_classes is not None:
            reset_class_head = True
            self.config.num_classes = num_classes
        if aspect_ratios is not None:
            reset_box_head = True
            self.config.aspect_ratios = aspect_ratios
        if num_scales is not None:
            reset_box_head = True
            self.config.num_scales = num_scales
        set_config_readonly(self.config)

        if reset_class_head:
            self.class_net = HeadNet(self.config, num_outputs=self.config.num_classes)
            for n, m in self.class_net.named_modules(prefix='class_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

        if reset_box_head:
            self.box_net = HeadNet(self.config, num_outputs=4)
            for n, m in self.box_net.named_modules(prefix='box_net'):
                if alternate_init:
                    _init_weight_alt(m, n)
                else:
                    _init_weight(m, n)

    @torch.jit.ignore()
    def toggle_head_bn_level_first(self):
        """ Toggle the head batchnorm layers between being access with feature_level first vs repeat
        """
        self.class_net.toggle_bn_level_first()
        self.box_net.toggle_bn_level_first()

    def forward(self, x, tab):
        x_b = self.backbone(x)
        x = self.fpn(x_b)
        x_class = self.class_net(x)
        x_box = self.box_net(x)
        x_classi_vis = self.vis_head(x_b[2])   #INSERTED CODE
        x_class_tab = self.tab(tab)            #INSERTED CODE
        x_classi = self.final_head(torch.cat([x_class_tab, x_classi_vis], dim=1))
        return x_class, x_box, x_classi #returns x_classi on top of original

In [87]:
# Used DetBenchTrain  
class DetClassiBenchTrain(nn.Module):
    def __init__(self, model, create_labeler=True):
        super(DetClassiBenchTrain, self).__init__()
        self.model = model
        self.config = model.config
        self.num_levels = model.config.num_levels
        self.num_classes = model.config.num_classes
        self.anchors = Anchors.from_config(model.config)
        self.max_detection_points = model.config.max_detection_points
        self.max_det_per_image = model.config.max_det_per_image
        self.soft_nms = model.config.soft_nms
        self.anchor_labeler = None
        if create_labeler:
            self.anchor_labeler = AnchorLabeler(self.anchors, self.num_classes, match_threshold=0.5)
        self.loss_fn = DetectionLoss(model.config)
        self.classi_loss = CrossEntropyLossFlat() #INSERTED CODE - loss_fn for classification loss

    def forward(self, x, target):
        tab = torch.stack(target['tab']).type(torch.LongTensor).to('cuda')
        class_out, box_out, classi_out = self.model(x, tab) #OUTPUTS INCLUDE classiicaiton predictions
        if self.anchor_labeler is None:
            # target should contain pre-computed anchor labels if labeler not present in bench
            assert 'label_num_positives' in target
            cls_targets = [target[f'label_cls_{l}'] for l in range(self.num_levels)]
            box_targets = [target[f'label_bbox_{l}'] for l in range(self.num_levels)]
            num_positives = target['label_num_positives']
        else:
            cls_targets, box_targets, num_positives = self.anchor_labeler.batch_label_anchors(
                target['bbox'], target['cls'])

        classi_targets = torch.stack(target['classi']).squeeze().type(torch.LongTensor).to('cuda')
        loss, class_loss, box_loss = self.loss_fn(class_out, box_out, cls_targets, box_targets, num_positives)
        
        #INSERTED CODE STARTS
        classi_loss = self.classi_loss(classi_out, classi_targets)
        loss += classi_loss
        #INSERTED CODE ENDS

        output = {'loss': loss, 'class_loss': class_loss, 'box_loss': box_loss, 'classi_pred':classi_out, 'classi_loss':classi_loss} #ADDED "classi_loss"

        if not self.training:
            # if eval mode, output detections for evaluation
            class_out_pp, box_out_pp, indices, classes = _post_process(
                class_out, box_out, num_levels=self.num_levels, num_classes=self.num_classes,
                max_detection_points=self.max_detection_points)
            output['detections'] = _batch_detection(
                x.shape[0], class_out_pp, box_out_pp, self.anchors.boxes, indices, classes,
                target['img_scale'], target['img_size'],
                max_det_per_image=self.max_det_per_image, soft_nms=self.soft_nms)
            
        return output

In [88]:
def create_model_m(model_name, bench_task='', num_classes=None, pretrained=False,
                 checkpoint_path='', checkpoint_ema=False, img_size=None, **kwargs):

    config = get_efficientdet_config(model_name)
    config.image_size = (img_size, img_size) if isinstance(img_size, int) else img_size

    return create_model_from_config_m(config, bench_task=bench_task, num_classes=num_classes, pretrained=pretrained,
                                      checkpoint_path=checkpoint_path, checkpoint_ema=checkpoint_ema, **kwargs)

In [89]:
def create_model_from_config_m(
        config, bench_task='', num_classes=None, pretrained=False,
        checkpoint_path='', checkpoint_ema=False, **kwargs):

    pretrained_backbone = kwargs.pop('pretrained_backbone', True)
    if pretrained or checkpoint_path:
        pretrained_backbone = False  # no point in loading backbone weights

    # Config overrides, override some config values via kwargs.
    overrides = (
        'redundant_bias', 'label_smoothing', 'legacy_focal', 'jit_loss', 'soft_nms', 'max_det_per_image', 'image_size')
    for ov in overrides:
        value = kwargs.pop(ov, None)
        if value is not None:
            setattr(config, ov, value)

    labeler = kwargs.pop('bench_labeler', False)

    # create the base model
    model = EfficientDetClassi(config, pretrained_backbone=pretrained_backbone, **kwargs)
    
    # pretrained weights are always spec'd for original config, load them before we change the model
    if pretrained:
        load_pretrained(model, config.url)

    # reset model head if num_classes doesn't match configs
    if num_classes is not None and num_classes != config.num_classes:
        model.reset_head(num_classes=num_classes)

    # load an argument specified training checkpoint
    if checkpoint_path:
        load_checkpoint(model, checkpoint_path, use_ema=checkpoint_ema)

    # wrap model in task specific training/prediction bench if set
    if bench_task == 'train':
        model = DetClassiBenchTrain(model, create_labeler=True)
    elif bench_task == 'predict':
        model = DetClassiBenchPredict(model)
    return model

In [90]:
def splitter(m):
    s = nn.Sequential(*m.model.children())
    return L(s[0], s[1:]).map(params)

In [91]:
class AvgMetric_m(Metric):
    "Average the values of `func` taking into account potential different batch sizes"
    def __init__(self, func):  self.func = func
    def reset(self):           self.total,self.count = 0.,0
    def accumulate(self, learn):
        bs = find_bs(torch.stack(learn.yb[0]['classi']).squeeze())
        self.total += learn.to_detach(self.func(learn.pred, *learn.yb))*bs
        self.count += bs
    @property
    def value(self): return self.total/self.count if self.count != 0 else None
    @property
    def name(self):  return self.func.func.__name__ if hasattr(self.func, 'func') else  self.func.__name__

In [92]:
def accuracy_m(inp, targ, axis=-1):
    "Compute accuracy with `targ` when `pred` is bs * n_classes"
    inp = inp['classi_pred']
    targ = torch.stack(targ['classi']).squeeze()
    pred,targ = flatten_check(inp.argmax(dim=axis), targ)
    return (pred == targ).float().mean()

In [93]:
@delegates(GradScaler)
class MixedPrecision_m(Callback):
    "Mixed precision training using Pytorch's `autocast` and `GradScaler`"
    order = 10
    def __init__(self, **kwargs): self.kwargs,self.autocast = kwargs,autocast()
    def before_fit(self): self.learn.scaler,self.scales = GradScaler(**self.kwargs),L()
    def before_batch(self): self.autocast.__enter__()
    def after_pred(self):
        #if listify(self.pred)[0].dtype==torch.float16: self.learn.pred = to_float(self.pred)
        for k in self.pred:
            if self.pred[k].dtype==torch.float16: self.pred[k] = to_float(self.pred[k])

    def after_loss(self): self.autocast.__exit__()
    def before_backward(self): self.learn.loss_grad = self.scaler.scale(self.loss_grad)
    def before_step(self):
        self.skipped=True
        self.scaler.step(self)
        if self.skipped: raise CancelStepException()
        self.scales.append(self.scaler.get_scale())
    def after_step(self): self.learn.scaler.update()

    @property # pretend to be an optimizer for `GradScaler`
    def param_groups(self): return self.opt.param_groups
    def step(self, *args, **kwargs): self.skipped=False

In [94]:
@patch
@delegates(GradScaler)
def to_fp16_m(self:Learner, **kwargs): return self.add_cb(MixedPrecision_m(**kwargs))

#Training

In [95]:
def get_learner(fold):
    t_dl, v_dl = get_dls(fold)
    bench = create_model_m('tf_efficientdet_d3_ap', 
                        'train',  
                        num_classes=2,
                        classi_class=4,
                        img_size=sz)
    
    model_type = icevision.models.ross.efficientdet
    metrics = [COCOMetric(metric_type=COCOMetricType.bbox), AvgMetric_m(accuracy_m)]
    learn = model_type.fastai.learner(dls=[t_dl, v_dl], 
                                      model=bench, 
                                      metrics=metrics,
                                      splitter = splitter,
                                      opt_func = ranger).to_fp16_m()
    return learn


In [96]:
for i in range(N_FOLDS):
    learn = get_learner(i)

    #learn.freeze()
    #learn.fit_flat_cos(3, 1e-3)
    MODEL_PATH = f'{path}/model/tf_efficientdet_d3_ap_{i}.pth'
    learn.model.load_state_dict(torch.load(MODEL_PATH))

    MODEL_OUTPUT_PATH = f'{path}/model/tf_efficientdet_d3_ap_{i}_2'
    sm = SaveModelCallback(fname=str(MODEL_OUTPUT_PATH))

    learn.unfreeze()
    learn.fit_flat_cos(3, 1e-4, cbs=sm)

  0%|          | 0/6334 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:136


  0%|          | 0/5067 [00:00<?, ?it/s]

AUTOFIX-SUCCESS - (record_id: 2873) - Clipping bbox xmin from -0.42773894736842105 to 0 (Before: <BBox (xmin:-0.42773894736842105, ymin:236.17508403669723, xmax:166.28560842105261, ymax:498.2087530275229)>) | icevision.utils.logger_utils:autofix_log:17


  0%|          | 0/1267 [00:00<?, ?it/s]

TypeError: __init__() got an unexpected keyword argument 'create_batch'